In [26]:
from pycparser import parse_file, c_ast

global parents
parents={}

In [34]:
def parse_c_file(file):
    ast = parse_file(file, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return ast

In [35]:
ast_graph = parse_c_file('./C files/if.c')

In [36]:
def computeParents(node, parent):
    global parents
    parents[node] = parent
    for (child, value) in node.children():
        computeParents(value, node) 

In [37]:
computeParents(ast_graph, None)

In [38]:
def findConditions(node):
    conditions = []
    if(type(node) == c_ast.While or type(node) == c_ast.If\
       or type(node) == c_ast.For):
        conditions.append(node.cond)
    for (child, value) in node.children():
        conditions += findConditions(value)
    return conditions

In [39]:
conditions = findConditions(ast_graph)

[BinaryOp(op='==',
         left=ID(name='b'
                 ),
         right=ID(name='a'
                  )
         ), BinaryOp(op='<',
         left=ID(name='b'
                 ),
         right=ID(name='a'
                  )
         )]
